In [1]:
from IPython.core.display import HTML
HTML("<style>.container { width: 98% }<\style>")

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [20]:
activities_train = pd.read_csv('../data/act_train.csv.zip', index_col='activity_id', dtype={'date': pd.Timestamp})
people = pd.read_csv('../data/people.csv.zip', index_col='people_id', dtype={'date': pd.Timestamp})

In [21]:
activities_train.head()

,people_id,date,activity_category,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10,outcome
activity_id,,,,,,,,,,,,,,
act2_1734928,ppl_100,2023-08-26,type 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 76,0
act2_2434093,ppl_100,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
act2_3404049,ppl_100,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
act2_3651215,ppl_100,2023-08-04,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
act2_4109017,ppl_100,2023-08-26,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0


In [5]:
people.head()

,char_1,group_1,char_2,date,char_3,char_4,char_5,char_6,char_7,char_8,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
people_id,,,,,,,,,,,,,,,,,,,,,
ppl_100,type 2,group 17304,type 2,2021-06-29,type 5,type 5,type 5,type 3,type 11,type 2,...,False,True,True,False,False,True,True,True,False,36
ppl_100002,type 2,group 8688,type 3,2021-01-06,type 28,type 9,type 5,type 3,type 11,type 2,...,False,True,True,True,True,True,True,True,False,76
ppl_100003,type 2,group 33592,type 3,2022-06-10,type 4,type 8,type 5,type 2,type 5,type 2,...,False,False,True,True,True,True,False,True,True,99
ppl_100004,type 2,group 22593,type 3,2022-07-20,type 40,type 25,type 9,type 4,type 16,type 2,...,True,True,True,True,True,True,True,True,True,76
ppl_100006,type 2,group 6534,type 3,2022-07-27,type 40,type 25,type 9,type 3,type 8,type 2,...,False,False,True,False,False,False,True,True,False,84


In [6]:
activities_train.index.str.slice(0, 4).value_counts()

act2    2039676
act1     157615
Name: activity_id, dtype: int64

In [7]:
activities_train['index_prefix'] = activities_train.index.str.slice(0, 4)

In [8]:
activities_train.groupby('index_prefix')['activity_category'].value_counts()

index_prefix  activity_category
act1          type 1               157615
act2          type 2               904683
              type 5               490710
              type 3               429408
              type 4               207465
              type 6                 4253
              type 7                 3157
Name: activity_category, dtype: int64

#### => type 1 activities associated only with act1 ids.

In [9]:
activities_train['index_suffix'] = activities_train.index.str.slice(start=5).astype(float).astype(int)

In [10]:
activities_train.outcome.value_counts()

0    1221794
1     975497
Name: outcome, dtype: int64

#### => Slightly unbalanced data set

In [19]:
activities_train.date.iloc[0]

'2023-08-26'